# Lead Scoring Classification

In [1]:
# importing libraries
import pandas as pd
import numpy as np

In [8]:
#read csv
df = pd.read_csv("./course_lead_scoring.csv")

In [12]:
# most frequent industry
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
education        187
healthcare       187
technology       179
manufacturing    174
Name: count, dtype: int64

## Correlation

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [25]:
df[['interaction_count','number_of_courses_viewed']].corrwith(df['lead_score'])

interaction_count           0.009888
number_of_courses_viewed   -0.004879
dtype: float64

In [26]:
df[['number_of_courses_viewed','annual_income']].corrwith(df['interaction_count'])

number_of_courses_viewed   -0.023565
annual_income               0.048618
dtype: float64

## spliting of dataset

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
df_full_train,df_test = train_test_split(df,test_size=0.2,random_state=42)
df_train,df_val = train_test_split(df_full_train,test_size=0.25,random_state=42)

In [34]:
df_full_train.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)
df_train.reset_index(drop=True,inplace=True)
df_val.reset_index(drop=True,inplace=True)

In [36]:
y_full_train = df_full_train['converted']
y_test = df_test['converted']
y_train = df_train['converted']
y_val = df_train['converted']

del df_full_train['converted']
del df_test['converted']
del df_train['converted']
del df_val['converted']

In [37]:
df_full_train['converted']

KeyError: 'converted'